In [7]:
import numpy as np
import pandas as pd
import time

In [8]:

file = 'EN'
path = './data/' + file
trainData =path + '/train'
testData =path + '/dev.in'
k =3
replaceWord ="#UNK#"

In [9]:
def emissionPara(xy,y):
    e_x_y= xy/y
    
    return e_x_y

def preprocess_training(data):
    start = time.process_time()   
    
    df= pd.read_csv(data, sep='/n', delimiter=None, names=['original'],index_col=False)
    # dropping null value columns to avoid errors 
    
    # new data frame with split value columns 
    new = df["original"].str.split(" ", n=1,expand=True) 

    # making separate first name column from new data frame 
    df["x"]= new[0] 

    # making separate last name column from new data frame 
    df["y"]= new[1] 
    return df

In [10]:
def uniqueCount(df,k,replaceWord):
    start = time.process_time()   
    x_dic={}

    # df display 
    uniqueX, uniqueCountX= np.unique(df['x'],return_counts=True)
    for i in range(len(uniqueX)):
        x_dic[uniqueX[i]] = uniqueCountX[i]
        
    
    for i, text in enumerate(df['x']):
        if x_dic[text] < k:
            df['x'][i] = replaceWord
#             df['original'][i]=df['original'][i].replace(text,replaceWord)
            df['original'][i]=df['x'][i] + ' ' + df['y'][i]
    y_dic={}
    uniqueY, uniqueCountY= np.unique(df['y'],return_counts=True)
    for i in range(len(uniqueY)):
        y_dic[uniqueY[i]] = uniqueCountY[i]
        
    xy_dic = {}
    df1= df.copy()
    # Get a tuple of unique values & their count from a numpy array
    df1.dropna(inplace = True) 
    uniqueXY, uniqueCountXY= np.unique(df1['original'].astype(str),return_counts=True)

    for i in range(len(uniqueXY)):
        xy_dic[uniqueXY[i]] = uniqueCountXY[i]
    # print('Unique Values : ', uniqueValues)
    # print('Count of Unique Values : ', uniqueCount)
    dft = pd.DataFrame([uniqueXY,uniqueCountXY]).T
    dft=dft.rename({0:'x_y',1:'count_x_y'},axis='columns')
    
    dft['count_y']=0
    for i,text in enumerate(dft['x_y']):
        data = text.split(" ")
        dft['count_y'][i]=y_dic[data[1]]
        
    end = time.time()
    print(str((time.process_time() - start)/60) + ' min')
    return dft
    


In [14]:
def xyPrediction(dft):    
    dft['emission']=emissionPara(dft['count_x_y'], dft['count_y'])
    
    dft1 = dft.copy()

    # new data frame with split value columns 
    new = dft1["x_y"].str.split(" ", n=1,expand=True) 

    # making separate first name column from new data frame 
    dft1["x"]= new[0] 

    # making separate last name column from new data frame 
    dft1["y"]= new[1] 
    
    xy_pred_dic = {}

    for word in dft1['x']:
        index = pd.Series.idxmax((dft1.loc[dft1['x'] == word]['emission']).astype(float))
        xy_pred_dic[word]=dft1['y'][index] 
#     print(time.process_time() - start)
    print(xy_pred_dic)
    return xy_pred_dic

In [15]:


def preprocess_test(data,k):
    start = time.process_time()   

    testdf1= pd.read_csv(data, sep='/n', delimiter=None, names=['original'],index_col=False)
    testdf= pd.read_csv(data, sep='/n', delimiter=None, names=['original'],index_col=False,skip_blank_lines=False)

    x_dic = {}

    uniqueX, uniqueCountX= np.unique(testdf1['original'],return_counts=True)
    for i in range(len(uniqueX)):
        x_dic[uniqueX[i]] = uniqueCountX[i]

    replaceWord ="#UNK#"

    testdf['modified']=''
#     print(testdf)
    for i, text in enumerate(testdf['original']):
    #         df['x'][i] = replaceWord
        try:
            if text not in xy_pred_dic:
            
                testdf['modified'][i]=testdf['original'][i].replace(text,replaceWord)
            else:
                testdf['modified'][i]=testdf['original'][i]
        except:
            continue
    testdf['predict_label']=''
    for index, word in enumerate(testdf['modified']):
#     print(word)
        try:
            testdf['predict_label'][index]= xy_pred_dic[word]
        except:
            continue
    print(str((time.process_time() - start)/60) + ' min')
    return testdf



In [16]:
predata = preprocess_training(trainData)


/mnt/c/Users/estee/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [17]:
countData=uniqueCount(predata,k,replaceWord)


/mnt/c/Users/estee/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1.00625 min


In [18]:
xy_pred_dic = xyPrediction(countData)

{'!': 'O', '#': 'B-ADJP', '#UNK#': 'B-INTJ', '$': 'B-NP', '%': 'I-ADJP', '&': 'I-NP', "'": 'B-NP', "''": 'O', "'80s": 'I-NP', "'S": 'B-VP', "'d": 'B-VP', "'ll": 'B-VP', "'m": 'B-VP', "'re": 'B-VP', "'s": 'B-NP', "'ve": 'B-VP', ',': 'O', '-': 'O', '--': 'O', '-LCB-': 'O', '-LRB-': 'O', '-RCB-': 'O', '-RRB-': 'O', '.': 'O', '...': 'O', '0.03': 'B-NP', '0.1': 'B-NP', '0.2': 'B-NP', '0.3': 'B-NP', '0.4': 'B-NP', '0.5': 'I-NP', '0.9': 'B-NP', '1': 'B-LST', '1,000': 'I-ADJP', '1,500': 'B-NP', '1.02': 'I-NP', '1.1': 'B-NP', '1.11': 'I-NP', '1.125': 'I-NP', '1.15': 'I-NP', '1.16': 'O', '1.19': 'I-NP', '1.2': 'I-NP', '1.22': 'I-NP', '1.25': 'I-NP', '1.29': 'I-NP', '1.3': 'I-NP', '1.32': 'I-NP', '1.375': 'I-NP', '1.4': 'I-NP', '1.42': 'I-NP', '1.44': 'I-NP', '1.5': 'I-NP', '1.50': 'I-NP', '1.6': 'I-NP', '1.7': 'I-NP', '1.70': 'I-NP', '1.71': 'I-NP', '1.8': 'I-NP', '1.8470': 'B-NP', '10': 'I-ADVP', '10,000': 'I-NP', '10-year': 'I-NP', '10.4': 'B-NP', '10.77': 'I-NP', '100': 'I-NP', '100,000': 'I-

In [10]:
testdf = preprocess_test(testData,k)

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


0.20260416666666667 min


In [11]:
testData = path + '/dev.out'
testresultdf= pd.read_csv(testData, sep='/n', delimiter=None, names=['original'],index_col=False)
testresultdf
new = testresultdf["original"].str.split(" ", n=1,expand=True) 

# making separate text column from new data frame 
testresultdf["x"]= new[0] 

# making separate label column from new data frame 
testresultdf["y"]= new[1] 
testresultdf

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,original,x,y
0,HBO B-NP,HBO,B-NP
1,has B-VP,has,B-VP
2,close B-NP,close,B-NP
3,to I-NP,to,I-NP
4,24 I-NP,24,I-NP
5,million I-NP,million,I-NP
6,subscribers I-NP,subscribers,I-NP
7,to B-PP,to,B-PP
8,its B-NP,its,B-NP
9,HBO I-NP,HBO,I-NP


In [12]:
final = pd.DataFrame()
final['result'] = testdf['modified'] + ' ' + testdf['predict_label']
final.head(5)

,result
0,HBO B-NP
1,has B-VP
2,close B-ADJP
3,to B-PP
4,24 I-NP


In [13]:

f = open(path+'/dev.p2.out','w')
for word in final['result']:
    f.write(word + '\n')
f.close()